<a href="https://colab.research.google.com/github/englissi/product/blob/main/(%ED%99%94%EC%83%81)Blue_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install gradio speechrecognition pydub gtts



In [14]:
import gradio as gr
import speech_recognition as sr
from pydub import AudioSegment
import os
from gtts import gTTS

def transcribe_audio(audio):
    recognizer = sr.Recognizer()
    audio = AudioSegment.from_file(audio)
    audio.export("temp.wav", format="wav")
    with sr.AudioFile("temp.wav") as source:
        audio_data = recognizer.record(source)
        try:
            text = recognizer.recognize_google(audio_data)
        except sr.UnknownValueError:
            text = "Sorry, I could not understand the audio."
        except sr.RequestError:
            text = "Sorry, there was an error with the speech recognition service."
    os.remove("temp.wav")
    return text

def speak_text(text):
    tts = gTTS(text)
    tts.save("question.mp3")
    return "question.mp3"

questions = [
    {"context": "These are based on the story Baby Animals. Now, baby dogs are called puppies. They like to run and they like to bark.", "question": "What else do they like to do?", "label": "Transcription of puppies' behavior", "answer": "They like to chew on shoes."},
    {"context": "Baby horses are called foals. Now they are very shy. If you go near them,", "question": "What will they do? Will they run away?", "label": "Transcription of foals' behavior", "answer": "They will run away."},
    {"context": "These are based on the story Where is he now?", "question": "Where is he now?", "label": "Transcription of current location", "answer": "He is in the hospital."},
    {"context": "He will go to the party together and they will even go to school together.", "question": "What else will they do together?", "label": "Transcription of future activities", "answer": "They will play soccer together."},
    {"context": "Do you like playing soccer?", "question": "Do you like playing soccer?", "label": "Transcription of personal preference", "answer": "No."},
    {"context": "These are based on the story My Best Friend, Snowflake. Look at the picture. She has a cat Snowflake. And Snowflake will do many things together. They will read books together. And they will drink milk together.", "question": "What else do they do together?", "label": "Transcription of shared activities", "answer": "They will take baths together."},
    {"context": "Do you like cats?", "question": "Do you like cats?", "label": "Transcription of personal preference", "answer": "Yes, Cats are cute like dogs."},
    {"context": "These are based on the story Camping is Not Fun. Now look at the boy. He will go camping with his family today. His family is excited but he is not excited.", "question": "Actually, how is he? How does he look?", "label": "Transcription of emotions", "answer": "He looks sad."},
    {"context": "What else does he want to do?", "question": "What else does he want to do?", "label": "Transcription of desires", "answer": "He wants to stay home."},
    {"context": "These are based on the story My dad is like Superman. Look at the page 50. This boy's dad is like superman. He helps his family at home. he helps his mom clean the house. He helps sis sister draw a picture.", "question": "Then how does his father help him?", "label": "Transcription of father's help", "answer": "He helps him do his homework."},
    {"context": "His dad helps people in town. He helps people cross the street. What else does his father help people in town?", "question": "What else does his father help people in town?", "label": "Transcription of father's help in town", "answer": ""}
]


current_question = 0
responses = []

def next_question():
    global current_question
    if current_question < len(questions):
        context = questions[current_question].get("context", "")
        question = questions[current_question]["question"]
        full_text = f"{context} {question}"
        question_audio = speak_text(full_text)
        current_question += 1
        return gr.update(value=question_audio, visible=True), gr.update(visible=True), questions[current_question-1]["label"], gr.update(visible=True), gr.update(visible=False)
    else:
        final_results = evaluate_responses()
        return gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(visible=False), gr.update(value=final_results, visible=True)

def save_response(audio):
    transcription = transcribe_audio(audio)
    responses.append(transcription)
    return transcription

def evaluate_responses():
    result = "<h2>Your Responses:</h2><br>"
    for i, question in enumerate(questions):
        user_response = responses[i] if i < len(responses) else "No response"
        result += f"<b>Q:</b> {question['question']}<br><b>Your Answer:</b> {user_response}<br><br>"
    return result

with gr.Blocks() as demo:
    gr.Markdown("### Interactive Questions")

    question_audio = gr.Audio(label="Question", visible=False)
    audio_input = gr.Audio(type="filepath", label="Your answer", visible=True)
    transcription_output = gr.Textbox(label="Transcription", visible=True)
    btn_next = gr.Button("Next", visible=True)
    final_output = gr.HTML(visible=False)

    def load_first_question():
        return next_question()

    demo.load(load_first_question, outputs=[question_audio, audio_input, transcription_output, btn_next, final_output])

    btn_next.click(next_question, outputs=[question_audio, audio_input, transcription_output, btn_next, final_output])
    audio_input.change(save_response, inputs=audio_input, outputs=transcription_output)

    demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://9d52b9aa6444722ea2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
